In one of my projects I have come across such task. Find the most common value of in string column in 30 minute windows, when we have one measurement per second. The length of data frame can be from 1h to any other bigger size.
One more thing was that the number of possible strings in the column is <20.

The propose solutions was based on pandas rolling and mode function from scipy. It was very slow, I have come with much faster solution which I want to show and compare with the rolling one in this notebook.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode
import time

In [2]:
# Let's create a data to show a problem
# I will create a data frame with  a random string column
# I will not evenly distributed strings in this column, it will be clear later why 
table1 = ["A", "B", "C", "D"]
table2 = ["V", "X", "Y", "Z"]
l_input = [table1[np.random.randint(0, 4)] for _ in range(5*3600)]+ \
          [table2[np.random.randint(0, 4)] for _ in range(5*3600)]
df_test = pd.DataFrame(data={"test": l_input})

task is to calculate the most common value in windows of size 1800

In [3]:
def RollingWay(df,column_name="test",window_size=1800):
    # first we have to change string to int beacuse rolling can not work with string columns
    # so we take all unique string and create dict for mapping
    all_str = np.sort(df[column_name].unique()) # sorting is to have predictable mapping 
    map_dict = {i: ix for ix, i in enumerate(all_str)}
    map_column_name ="mapped_" +column_name
    df[map_column_name] = df[column_name].map(map_dict)
    # finall rolling
    df["results"] = df[map_column_name].rolling(window=window_size, min_periods=window_size, center=True).\
                                        apply(lambda x: (mode(x)[0]), raw=True)
    return df["results"]
    

In [4]:
t = time.process_time()
v1=RollingWay(df_test)
elapsed_time = time.process_time() - t
print(elapsed_time)

4.502422888


I think that the biggest problem of the rolling approach is that step n does not use any info from step n-1.
The difference between those two steps are is in two values.
I have coded function using that to check how faster it will be.


In [5]:
def MyWay(df,column_name="test",window_size=1800):
    # as first we also have to get all unique values
    all_str = np.sort(df[column_name].unique())
    # the we get results for the first window
    data_dict = df[column_name][: window_size].value_counts().to_dict()
    # creates a dictionary for the first window which will include all strings in column 
    # this is why evenly distributed strings where used
    data_dict.update({i:0 for i in all_str if i not in data_dict.keys()})
    # get the list which will store results
    results = list()
    # put results for first window 
    results.append(max(data_dict, key=data_dict.get))
     #in each step we removing count for the low index and add count from the up index 
    for iplus, iminus in zip(df[column_name][window_size:].values, df[column_name][:-1*window_size].values):
        data_dict[iplus] = data_dict[iplus]+1
        data_dict[iminus] = data_dict[iminus]-1
        # add we get results by finding the most common value
        results.append(max(data_dict, key=data_dict.get))
    return np.array(results)


In [6]:
t = time.process_time()
v2=MyWay(df_test)
elapsed_time = time.process_time() - t
print(elapsed_time)

0.06876973699999933


My way is much faster :)